# Προχωρημένες Υπολογιστικές Μέθοδοι Ι
## Homework 1b - Gauss Siedel Method
### Synetos Giorgos - Aravanis Dimitris

In [1]:
#-------------------------------
#Import Library & Settings
#-------------------------------
import numpy as np
import math
#from IPython.core.display import display, HTML
from IPython.display import display, Markdown, Latex
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline

# Poisson equation

$ \bigtriangledown ^2  \Phi =4 $

subject to B.C:  $ Φ(x,y) = x^2 + y^2 $



### Grid Definition
For the present demonstration  we are considering a grid of 5x5. 

However the grid can be adjsuted.

In [2]:
#-------------------------------
#Set the Grid
#-------------------------------

L, H = 1,1  #Length,Height
IM, JM = 4, 4 #Number of columns & rows
Dx, Dy = L/(IM-1) , H/(JM-1)
a = Dx/Dy 
f = 4 # Η τιμή απο τη εξίσωση Poison που καλούμαστε να λύσουμε

# Set array size and set the interior value with v_init
v_init = 0 #initial guess for interior nodes
v = np.empty((JM, IM))
v_new = np.empty((JM, IM))
A = np.empty((JM, IM))

b = np.empty(IM*JM)
#b.fill(f)
v.fill(v_init)

### Analytical Solution {u}

In [3]:
u = np.empty((JM, IM))
for i in range(JM):
    for j in range(IM):
        u[i, j] = (j*Dx)**2 + (i*Dy)**2
uk = u.flatten()

### Boundary Conditions

In [4]:
#-------------------------------
#Set Boundary conditions
#-------------------------------

#Top Boundary
bc_top = np.empty(IM)
for i in range(IM):
    bc_top[i] = (i*Dx)**2 + ((JM-1)*Dy)**2
v[(JM-1):, :] = bc_top

  
#Bottom Boundary
bc_bottom = np.empty(IM)
for i in range(IM):
    bc_bottom[i] = (i*Dx)**2
v[:1, :] = bc_bottom


#Right Boundary
bc_right = np.empty(JM)
for i in range(JM):
    bc_right[i] = (i*Dy)**2 + ((IM-1)*Dx)**2
bc_right = bc_right[np.newaxis, :].T
v[:, (IM-1):] = bc_right

#Left Boundary
bc_left = np.empty(JM)
for i in range(0, JM):
    bc_left[i] = (i*Dy)**2
bc_left = bc_left[np.newaxis, :].T
v[:, :1] = bc_left 

vk = v.flatten()


### Boundary Nodes

In [5]:
BN = list()

#Nodes located on Bottom
for k in range (IM):
    BN.append(k)
#Nodes located on Top
for k in range ((IM*(JM-1)),IM*JM):
    BN.append(k)
#Nodes located on the Left Side excluding first and last node
for i in range (1,JM-1):
    k = i*(IM)
    BN.append(k)
#Nodes located on the Right Side excluding first and last node
for i in range (1,JM-1):
    k = (i+1)*IM-1
    BN.append(k)
BN = sorted(BN)
BN

[0, 1, 2, 3, 4, 7, 8, 11, 12, 13, 14, 15]

### Vector b


In [6]:
for k in range (IM*JM):
    if k not in BN:
        b[k]=4
b

array([  0.00000000e+000,   0.00000000e+000,   2.12212537e-314,
         4.94065646e-324,   0.00000000e+000,   4.00000000e+000,
         4.00000000e+000,   2.15550246e-314,   0.00000000e+000,
         4.00000000e+000,   4.00000000e+000,   0.00000000e+000,
         0.00000000e+000,  -5.22433787e-110,   2.15076527e-314,
         2.20312697e-314])

### Matrix A

In [7]:
dim = (IM*JM,IM*JM)
A = np.zeros(dim)

#Boundary Nodes A[i,i]=1
for index in BN:
    A[index,index] = 1

    
#Inner Nodes
for k in range (IM*JM):
    if k not in BN:
        A[k,k]= -2*(Dx**2+Dy**2)/((Dx**2) * (Dy**2))
        vertical_neighbours = [k+IM,k-IM]
        for v_neighbour in vertical_neighbours:
            A[k,v_neighbour]= Dx**2/((Dx**2) * (Dy**2))
        horizontal_neighbours = [k+1,k-1]
        for h_neighbour in horizontal_neighbours:
            A[k,h_neighbour]= Dy**2/((Dx**2) * (Dy**2))



### Matrix A Decomposition
[A]{x} = {b} is transformed into ( [D] - [L] - [U] ){x}=b

In [8]:
D = np.diag(np.diag(A))
L = np.tril(A,-1)
U = np.triu(A,1)

### Further Simplifications
Define T = inv.D (L+U)
and    c = inv.D {b}
Jacobi method is written as
{x_new} = [T]{x_old}+[c]

In [9]:
T = np.dot(np.linalg.inv(D),(L+U))
c = np.dot(np.linalg.inv(D),b)

### Conditional Number

In [10]:
def conditional_number(matrix):
    inv_matrix = np.linalg.inv(matrix)
    #print(np.linalg.norm(matrix), np.linalg.norm(inv_matrix))
    cond_a = np.linalg.norm(matrix) * np.linalg.norm(inv_matrix)
    return cond_a

cond_a = conditional_number(A)
print(cond_a)

288.385069932


# New Attempt for Jacobi

In [11]:
# n=len(b)

# x_old = vk[:]
# x = np.empty(n)

# def jacobi(a,x,x_old,b,tol=10**-6,n_iter=10):
#     for k in range(n_iter):
#         #print("iteration",k)
#         #if np.linalg.norm(x - x_old) < tol:
#         #    break

#         for i in range(n):
#             if i not in BN:
#                 x[i] = 1.0 / a[i, i] * sum(-a[i, j] * x_old[j] + b[i] for j in range(n) if j != i)
#         print("x_old= \n",x_old)
#         print("x_new= \n", x)
#         x_old = x[:]
#     return x
        
# test=jacobi(A,x,x_old,b)
# test


### Jacobi Method


In [12]:
Jacobi = "yes"
if Jacobi == "yes":
    #-------------------------------
    #Jacobi Method
    #-------------------------------

    v_newk = vk[:]
    counter= list()
    residual = list()
    e = list()
    error = list()
    res=1
    iteration = 0
    while res > 10**-6 and iteration in range(0, 1000) :
        r=0
        vk[:]=v_newk
        for i in range(1, JM-1):
            for j in range(1, IM-1):
                k= i*IM + j
                v_newk[k] = (-A[k][k+IM]*vk[k+IM] - A[k][k-IM]*vk[k-IM]- A[k][k+1]*vk[k+1] - A[k][k-1]*vk[k-1] + b[k] )/ A[k][k]


                r = r + b[k]-np.dot(A[k,k],v_newk[k])
        res = np.linalg.norm(r)

        #Convergence data
        counter.append(iteration)
        residual.append(res)
        iteration += 1

    print("After",iteration," iterations,",
          "the residual on the final iteration is ",res)

uk - v_newk

After 1000  iterations, the residual on the final iteration is  96.0


array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.])

In [13]:
# Jacobi = "yes"
# if Jacobi == "yes":
#     #-------------------------------
#     #Jacobi Method
#     #-------------------------------

#     v_new[:] = v
#     counter= list()
#     residual = list()
#     e = list()
#     error = list()
#     res=1
#     iteration = 0
#     while res > 10**-6 and iteration in range(0, 100) :
#         v[:]=v_new
#         for i in range(1, JM-1):
#             for j in range(1, IM-1):
#                 k= i*IM + j
#                 v_new[i, j] = ((A[k][k+IM]*v[i+1][j] + A[k][k-IM]*v[i-1][j] \
#                                 + (A[k][k+1]*v[i][j+1] + A[k][k-1]*v[i][j-1]))-b[k])/-A[k][k]

                
#         v_newk = v_new.flatten()
#         r = b-np.dot(A,v_newk)
#         res = np.linalg.norm(r)

#         #Error
#         e = (uk - v_newk)
#         error.append(np.linalg.norm(e))

#         #Convergence data
#         counter.append(iteration)
#         residual.append(res)
#         iteration += 1

#         #Print unequality 3 of the notes
#         uneq_1 = ((1/cond_a) * np.linalg.norm(res)) / np.linalg.norm(b)
#         uneq_2 = np.linalg.norm(e) / np.linalg.norm(v_newk)
#         uneq_3 = (cond_a * np.linalg.norm(res)) / np.linalg.norm(b)
#         #print(uneq_1, "=<", uneq_2, "=<", uneq_3)

#     print("After",iteration," iterations,",
#           "the residual on the final iteration is ",res)


# Kiusalaas - Gauss Siedel Method


In [14]:
# def gaussSeidel(iterEqs,x,tol = 1.0e-6):
#     omega = 1.0
#     k = 10
#     p = 1
#     for i in range(1,501):
#         xOld = x.copy()
#         x = iterEqs(x,omega)
#         dx = math.sqrt(np.dot(x-xOld, x-xOld))
#         if dx < tol:
#             return x,i,omega

# def iterEqs(x,omega):
    

## Plots

In the following figures the counter lines and the convergence are presented.

#### Counter Lines

In [15]:
plotcontourlines = "yes"

if plotcontourlines == "yes":
    # Create Grid
    X, Y = np.meshgrid(np.arange(0, L, L/IM), np.arange(0, H, H/JM))
    # Configure the contour
    cp = plt.contour(X, Y, v_new)
    plt.clabel(cp, inline=True, fontsize=10)
    plt.title("Contour Lines")
    plt.xlabel('x-axis')
    plt.ylabel('y-axis')


    # Set Colorbar
    plt.colorbar()

    # Show the result in the plot window
    plt.show()
    
else:
    pass

/Users/gsynetos/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in true_divide


TypeError: only length-1 arrays can be converted to Python scalars

#### Convergence Plot

In [ ]:
# plotconvergence = "no"

# if plotconvergence == "yes":
#     #plot convergence
#     plt.plot(counter, residual)
#     plt.yscale('log')
#     plt.title('Convergence')
#     plt.xlabel('iterations')
#     plt.ylabel('residual log_10')
#     plt.grid(True)
#     plt.show()
    
# else:
#     pass

### Error Plot

In [ ]:
# ploterror = "no"

# if ploterror == "yes":
#     #plot error
#     plt.plot(counter, error)
#     plt.title('error')
#     plt.yscale('log')
#     plt.xlabel('iterations')
#     plt.ylabel('error')
#     plt.grid(True)
#     plt.show()
    
# else:
#     pass